# Visualizations

In [1]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [2]:
depression = pd.read_csv("Deepression.csv")
depression.head()

,Number,Sleep,Appetite,Interest,Fatigue,Worthlessness,Concentration,Agitation,Suicidal Ideation,Sleep Disturbance,Aggression,Panic Attacks,Hopelessness,Restlessness,Low Energy,Depression State
0,1,1.0,1.0,1.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,Mild
1,2,2.0,5.0,5.0,1.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,Moderate
2,3,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Severe
3,4,1.0,1.0,1.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,No depression
4,5,2.0,5.0,5.0,1.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,Moderate


In [3]:
depression.dtypes

Number                 int64
Sleep                float64
Appetite             float64
Interest             float64
Fatigue              float64
Worthlessness        float64
Concentration        float64
Agitation            float64
Suicidal Ideation    float64
Sleep Disturbance    float64
Aggression           float64
Panic Attacks        float64
Hopelessness         float64
Restlessness         float64
Low Energy           float64
Depression State      object
dtype: object

In [4]:
missing_values = depression.isna().sum()
print("Missing values:")
print(missing_values)

Missing values:
Number                 0
Sleep                273
Appetite             273
Interest             273
Fatigue              273
Worthlessness        273
Concentration        273
Agitation            273
Suicidal Ideation    273
Sleep Disturbance    273
Aggression           273
Panic Attacks        273
Hopelessness         273
Restlessness         273
Low Energy           273
Depression State     273
dtype: int64


In [5]:
mode = depression['Depression State'].mode()[0]
depression['Depression State'].fillna(mode, inplace = True)

num_cols = [
    'Sleep' , 'Appetite', 'Interest','Fatigue','Worthlessness',
    'Concentration','Agitation','Suicidal Ideation','Sleep Disturbance','Aggression',
    'Panic Attacks','Hopelessness','Restlessness','Low Energy'
]
depression[num_cols] = depression[num_cols].apply(lambda x: x.fillna(x.mean()))

In [6]:
depression.isna().sum()

Number               0
Sleep                0
Appetite             0
Interest             0
Fatigue              0
Worthlessness        0
Concentration        0
Agitation            0
Suicidal Ideation    0
Sleep Disturbance    0
Aggression           0
Panic Attacks        0
Hopelessness         0
Restlessness         0
Low Energy           0
Depression State     0
dtype: int64

In [11]:
print(depression['Depression State'].unique())

['Mild' 'Moderate' 'Severe' 'No depression' '2No depression'
 '5No depression']


In [12]:
depression['Depression State'] = (
    depression['Depression State']
    .str.strip()
    .str.replace(r'^\d+', '', regex = True)
    .str.strip()
)
print(depression['Depression State'].unique())

['Mild' 'Moderate' 'Severe' 'No depression']


In [19]:
app = dash.Dash(__name__)
app.title = 'Depression Dashboard'

app.layout = html.Div([
    html.H1('Depression Dashboard'),
    
    dcc.Dropdown(
        id = 'statedropdown',
        options=[{'label': state, 'value': state} for state in depression['Depression State'].unique()],
        value = depression['Depression State'].unique()[0]
    ),
    
    dcc.Dropdown(
        id = 'combodropdown',
        options = [
            {'label' : 'Mild and Moderate', 'value':'Mild,Moderate'},
            {'label' : 'Mild and Severe', 'value' : 'Mild,Severe'},
            {'label' : 'Mild and No depression', 'value' : 'Mild,No depression'},
            {'label' : 'Moderate and Severe', 'value' : 'Moderate,Severe'},
            {'label' : 'Moderate and No depression', 'value' : 'Moderate,No depression'},
            {'label' : 'Severe and No depression', 'value' : 'Severe,No depression'}
        
        ],
        value='Mild,Moderate'
    ),
    
    html.Div([
        dcc.Graph(id = 'box-plot'),
        dcc.Graph(id = 'histogram'),
        dcc.Graph(id = 'heatmap'),
        dcc.Graph(id = 'pie-chart'),
        dcc.Graph(id = 'bubble-plot'),
        dcc.Graph(id = 'stacked-bar'),
        dcc.Graph(id = 'density-plot'),
        dcc.Graph(id = 'three-d-scatter')
    ])
])

@app.callback(
    [Output('box-plot', 'figure'),
     Output('histogram', 'figure'),
     Output('heatmap', 'figure'),
     Output('pie-chart', 'figure'),
     Output('bubble-plot', 'figure'),
     Output('stacked-bar', 'figure'),
     Output('density-plot','figure'),
     Output('three-d-scatter', 'figure')],
    [Input('statedropdown', 'value'),
    Input('combodropdown', 'value')]
)
def visualizations(selected_state,selected_combo):
    selected_states = selected_combo.split(',')
    depression_data = depression[depression['Depression State'].isin(selected_states + [selected_state])]

    box_fig = go.Figure()
    for col in num_cols:
        box_fig.add_trace(go.Box(y = depression_data[col], name = col))
    box_fig.update_layout(title = 'Box Plot of Numeric Features')

    hist_fig = go.Figure()
    for col in num_cols:
        hist_fig.add_trace(go.Histogram(x = depression_data[col], name = col, opacity = 0.75))
    hist_fig.update_layout(title = 'Histogram of Numeric Features', barmode = 'overlay')

    correlation_matrix = depression_data[num_cols].corr()
    heatmap_fig = px.imshow(correlation_matrix, text_auto = True, color_continuous_scale = 'Viridis')
    heatmap_fig.update_layout(title = 'Heatmap of Feature Correlations')

    pie_fig = px.pie(depression_data, names = 'Depression State', title = 'Distribution of Depression States')
    pie_fig.update_layout(title = 'Pie Chart of Depression States')
    
    bubble_fig = px.scatter(depression_data, x = 'Sleep', y = 'Fatigue', size = 'Low Energy', color = 'Depression State',
                           hover_name='Depression State', size_max=60)
    bubble_fig.update_layout(title='Bubble Plot of Sleep and Fatigue with Low Energy')

    agg_data = depression.groupby('Depression State').sum().reset_index()
    stacked_bar_fig = px.bar(agg_data, x='Depression State', y=num_cols,
                             title='Stacked Bar Chart of Numeric Features by Depression State',
                             labels={'value': 'Sum of Features', 'variable': 'Feature'},
                             height=500)
    stacked_bar_fig.update_layout(barmode='stack')
    
    density_fig = px.density_contour(depression_data, x = 'Sleep', y = 'Fatigue', color = 'Depression State',
                                     title = 'Density Plot of Sleep vs. Fatigue')
    density_fig.update_layout(title = 'Density Plot of Sleep vs. Fatigue')
    
    three_d_fig = px.scatter_3d(depression_data, x = 'Sleep', y = 'Fatigue', z = 'Low Energy', color = 'Depression State')
    three_d_fig.update_layout(title = '3D Scatter Plot of Sleep, Fatigue, and Low Energy')


    return box_fig, hist_fig, heatmap_fig, pie_fig, bubble_fig, stacked_bar_fig,density_fig,three_d_fig

if __name__ == '__main__':
    app.run_server(debug = True)

# References : (https://www.kaggle.com/datasets/hamjashaikh/mental-health-detection-dataset) (https://dash.plotly.com/tutorial)